<a href="https://colab.research.google.com/github/piotr-sobieraj/trurl/blob/main/trurl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Załadowanie modelu

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Nazwa modelu na Hugging Face
MODEL_NAME = "speakleash/Bielik-11B-v2.3-Instruct"

# Ładowanie tokenizera i modelu
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/27.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

# Definicje generatora wierszy i słów na 'n'

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"  # Automatyczny wybór GPU lub CPU

# Przeniesienie modelu na GPU z mniejszą precyzją
model = model.to(device, dtype=torch.bfloat16)


def generate_text(messages, max_new_tokens=100):
    """Generuje tekst na podstawie podanych wiadomości."""
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    generated_ids = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Usunięcie prefiksów system/user
    output_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Usunięcie pierwszych 4 linii z generowanego tekstu - śmieci
    output_lines = output_text.split("\n")[4:]
    filtered_output = "\n".join(output_lines).strip()

    return filtered_output

def generate_trurl_poem():
    """Generuje wiersz w stylu Trurla."""
    messages = [
        {"role": "system", "content": "Twórz zabawne i absurdalne rymowane wiersze w stylu Trurla z Cyberiady, pełne technologicznych metafor. Nie więcej niż 4 linijki"},
        {"role": "user", "content": "Napisz wiersz w stylu Trurla o tematyce komputerowej, związanej z matematyką."}
    ]
    return generate_text(messages, max_new_tokens=200)


def generate_object_that_start_with_n():
    """Generuje słowo zaczynające się na literę n."""
    messages = [
        {"role": "system", "content": "Twórz zabawne i absurdalne neologizmy w stylu Trurla z Cyberiady, zaczynające się na literę N"},
        {"role": "user", "content": "Wygeneruj zabawny neologizm na literę n związany z życiem, filozofią i matematyką, maksymalnie 20 słów. Podaj nazwę i definicję"}
    ]
    return generate_text(messages, max_new_tokens=500)


# Definicja parsera

In [4]:
import re

def parse_command(command: str):
    """Parser dla poleceń KONSTRUKTOR/DAJ."""

    # Obsługa wielu linii – znajdź wszystkie konstrukcje
    constructor_matches = re.findall(r"KONSTRUKTOR\s+(\w+):\s+(\w+)\(\)", command)
    request_matches = re.findall(r"DAJ\s+(\w+)", command)

    objects = {}

    # Obsługa konstruktorów
    for obj_type, obj_name in constructor_matches:
        if obj_name == "liryk":
            objects[obj_type] = generate_trurl_poem()  # Tworzymy obiekt w słowniku
            continue  # Przeskakujemy resztę kodu, żeby nie nadpisać
        elif obj_name == "coś_na_n":
            objects[obj_type] = generate_object_that_start_with_n()
            continue  # Przeskakujemy resztę kodu
        else:
            objects[obj_type] = f"Nieznany konstruktor: {obj_name}"

    # Obsługa żądań "DAJ"
    results = []
    for obj_name in request_matches:
        if obj_name in objects:
            results.append(objects[obj_name])  # Pobieramy obiekt
        else:
            results.append(f"Nieznane polecenie dla: {obj_name}")

    return "\n".join(results)

# Przypadki użycia

In [7]:
# Test parsera
trurl_command_liryk = """KONSTRUKTOR wiersz: liryk()
                         DAJ wiersz"""

print(parse_command(trurl_command_liryk))

**Pół kilo liczb na półmisku**
To cyfry cud,
Gdy je pomielm w algorytmach młynku,
Kwadraty licz w prawdy ściśle wszczepionych:
Matematyka, królowo moich snów!


In [8]:
trurl_command_cos = """KONSTRUKTOR słowo: coś_na_n()
                       DAJ słowo"""
print(parse_command(trurl_command_cos))

### Neologizm: Niedocenialność

Definicja: **Niedocenialność** to unikalna zdolność do dostrzegania ulotnych, metafizycznych prawd i jednoczesnego ich umniejszania. Jest to paradoksalne połączenie mądrości i autoironii, które pozwala nam rozwijać się duchowo, będąc jednocześnie w ciągłej pogoni za własną skromnością.

Niedocenialność objawia się w sytuacjach, w których jednostka dostrzega, że jej umiejętności matematyczne przewyższają zdolności innych, ale mimo to, zamiast popaść w samozachwyt, wykorzystuje ten dar do rozwijania umiejętności miękkich, takich jak empatia czy sztuka negocjacji.

W skrócie, Niedocenialność to próba łączenia w jednej osobie Sokratesa, Euklidesa i Humpty-Dumpty'ego w jednym zabawnym, absurdalnym tańcu.
